# Mobile Get Input Notebook - Phase 1

**Phase 1 Improvements**: Consolidated configuration & improved helper functions for clarity and reliability.

This notebook automates creation of terrain profiles for ITU-R P.1812-6 propagation prediction.

## Workflow
1. **Setup**: Configure transmitter location and parameters
2. **Generate Receivers**: Create uniformly distributed receiver points
3. **Extract Profiles**: Get terrain elevation and land cover data (via module)
4. **Export**: Save profiles to CSV for batch processing

## Imports

In [1]:
import os
import math
import sys
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from dataclasses import dataclass
from typing import Iterable, Union

# Find and add project root to path
possible_roots = [
    Path.cwd(),
    Path.cwd().parent,
    Path.cwd().parent.parent,
]

project_root = None
for root in possible_roots:
    if (root / 'config_sentinel_hub.py').exists():
        project_root = root
        break

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import Sentinel Hub config
import requests
from config_sentinel_hub import (
    SH_CLIENT_ID, SH_CLIENT_SECRET,
    TOKEN_URL, PROCESS_URL, COLLECTION_ID,
)
from rasterio.io import MemoryFile
import rasterio
import rasterio.transform
import elevation

/Users/oz/Documents/mst_gis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Path Setup

In [2]:
# Detect project root by looking for src/ or config_sentinel_hub.py
notebook_dir = Path.cwd()

# Search for project root
project_root = None
for candidate in [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]:
    if (candidate / 'src').exists() or (candidate / 'config_sentinel_hub.py').exists():
        project_root = candidate
        break

if not project_root:
    project_root = Path.cwd()  # Fallback to current dir

profiles_dir = project_root / 'data' / 'input' / 'profiles'
api_data_dir = project_root / 'data' / 'intermediate' / 'api_data'
workflow_dir = project_root / 'data' / 'intermediate' / 'workflow'

profiles_dir.mkdir(parents=True, exist_ok=True)
api_data_dir.mkdir(parents=True, exist_ok=True)
workflow_dir.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Profiles dir: {profiles_dir}")

Project root: /Users/oz/Documents/mst_gis
Profiles dir: /Users/oz/Documents/mst_gis/data/input/profiles


## Configuration

In [3]:
# ============================================================================
# CONFIGURATION - Edit these parameters for your simulation
# ============================================================================

CONFIG = {
    'TRANSMITTER': {
        'tx_id': 'TX_0001',
        'longitude': -13.40694,
        'latitude': 9.345,
        'antenna_height_tx': 57,
        'antenna_height_rx': 10,
    },
    'P1812': {
        'frequency_ghz': 0.9,
        'time_percentage': 50,
        'polarization': 1,
    },
    'RECEIVER_GENERATION': {
        'max_distance_km': 11,
        'azimuth_step': 10,
        'distance_step': 0.5,
        'sampling_resolution': 30,
    },
    'SENTINEL_HUB': {
        'buffer_m': 11000,
        'chip_px': 734,
        'year': 2020,
    },
    'LCM10_TO_CT': {
        100: 1, 80: 2, 30: 2, 40: 2, 70: 2, 110: 2, 254: 2,
        20: 3, 50: 3, 10: 4, 60: 4, 90: 4,
    },
    'CT_TO_R': {1: 0, 2: 0, 3: 10, 4: 15, 5: 20},
}

# Derived values
tx_lon = CONFIG['TRANSMITTER']['longitude']
tx_lat = CONFIG['TRANSMITTER']['latitude']
max_distance_km = CONFIG['RECEIVER_GENERATION']['max_distance_km']
n_points = int(max_distance_km * 1000 / CONFIG['RECEIVER_GENERATION']['sampling_resolution'])
azimuths = list(range(0, 360, CONFIG['RECEIVER_GENERATION']['azimuth_step']))
distances = np.arange(
    CONFIG['RECEIVER_GENERATION']['distance_step'],
    max_distance_km + CONFIG['RECEIVER_GENERATION']['distance_step'],
    CONFIG['RECEIVER_GENERATION']['distance_step']
)

print(f"Transmitter: ({tx_lat}, {tx_lon})")
print(f"Azimuths: {len(azimuths)} | Profile points: {n_points}")

Transmitter: (9.345, -13.40694)
Azimuths: 36 | Profile points: 366


## Transmitter Definition

In [4]:
@dataclass
class Transmitter:
    tx_id: str
    lon: float
    lat: float
    htg: float
    f: float
    pol: int
    p: float
    hrg: float

tx = Transmitter(
    tx_id=CONFIG['TRANSMITTER']['tx_id'],
    lon=CONFIG['TRANSMITTER']['longitude'],
    lat=CONFIG['TRANSMITTER']['latitude'],
    htg=CONFIG['TRANSMITTER']['antenna_height_tx'],
    f=CONFIG['P1812']['frequency_ghz'],
    pol=CONFIG['P1812']['polarization'],
    p=CONFIG['P1812']['time_percentage'],
    hrg=CONFIG['TRANSMITTER']['antenna_height_rx'],
)
print(tx)

Transmitter(tx_id='TX_0001', lon=-13.40694, lat=9.345, htg=57, f=0.9, pol=1, p=50, hrg=10)


## Helper Functions

Define functions for credential resolution, land cover, and profile extraction.

In [5]:
def resolve_credentials(verbose: bool = True, fallback_id=None, fallback_secret=None):
    """Get Sentinel Hub credentials from environment or fallback."""
    env_id = os.environ.get("SH_CLIENT_ID", "").strip()
    env_secret = os.environ.get("SH_CLIENT_SECRET", "").strip()

    if env_id and env_secret:
        if verbose:
            print("  ✓ Found credentials in environment variables")
        return env_id, env_secret

    const_id = (fallback_id or SH_CLIENT_ID or "").strip()
    const_secret = (fallback_secret or SH_CLIENT_SECRET or "").strip()

    if not const_id or not const_secret or "REPLACE_ME" in str(const_id) or "REPLACE_ME" in str(const_secret):
        raise RuntimeError("Credentials not found. Set SH_CLIENT_ID and SH_CLIENT_SECRET.")

    if verbose:
        print("  ✓ Found credentials in config file")
    return const_id, const_secret


def meters_to_deg(lat, meters):
    dlat = meters / 111_320.0
    dlon = meters / (111_320.0 * math.cos(math.radians(lat)))
    return dlat, dlon


def get_token(client_id, client_secret):
    r = requests.post(TOKEN_URL, data={
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
    }, timeout=30)
    r.raise_for_status()
    return r.json().get("access_token")


def landcover_at_point(client_id, client_secret, lat, lon, token_url, process_url, collection_id, year=2020, buffer_m=1000, chip_px=32, save_path=None):
    token = get_token(client_id, client_secret)
    dlat, dlon = meters_to_deg(lat, buffer_m)
    bbox = [lon - dlon, lat - dlat, lon + dlon, lat + dlat]

    evalscript = """//VERSION=3
function setup() { return {input: [\"LCM10\"], output: {bands: 1, sampleType: \"UINT8\"}};}function evaluatePixel(s) { return [s.LCM10];}"""

    body = {
        "input": {"bounds": {"bbox": bbox, "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"}},
                   "data": [{"type": f"byoc-{collection_id}", "dataFilter": {"timeRange": {"from": f"{year}-01-01T00:00:00Z", "to": f"{year}-12-31T23:59:59Z"}}}]},
        "output": {"width": chip_px, "height": chip_px, "responses": [{"identifier": "default", "format": {"type": "image/tiff"}}]},
        "evalscript": evalscript,
    }

    r = requests.post(process_url, json=body, headers={
        "Authorization": f"Bearer {token}", "Content-Type": "application/json", "Accept": "image/tiff"},
        timeout=120)
    r.raise_for_status()

    if save_path:
        with open(save_path, "wb") as f:
            f.write(r.content)

    with MemoryFile(r.content) as memfile:
        with memfile.open() as ds:
            arr = ds.read(1)

    center_code = int(arr[arr.shape[0] // 2, arr.shape[1] // 2])
    return center_code, arr


def generate_profile_points(tx_lon, tx_lat, max_distance_km, n_points, azimuth_deg, tif_path, lcm10_to_ct, ct_to_r, zones_path=None):
    # NOTE: elevation.seed() is called ONCE before the loop in extract_profiles cell
    # DO NOT call it here to avoid redundant calls on every iteration
    tx_gdf = gpd.GeoDataFrame(geometry=[Point(tx_lon, tx_lat)], crs="EPSG:4326")
    utm_crs = tx_gdf.estimate_utm_crs()
    tx_utm = tx_gdf.to_crs(utm_crs)
    center = tx_utm.geometry.iloc[0]

    max_m = max_distance_km * 1000.0
    step_m = max_m / (n_points - 1)
    theta = math.radians(azimuth_deg)
    dx_unit, dy_unit = math.sin(theta), math.cos(theta)

    points_utm = [Point(center.x + i * step_m * dx_unit, center.y + i * step_m * dy_unit) for i in range(n_points)]
    distances_km = [i * step_m / 1000.0 for i in range(n_points)]

    gdf_utm = gpd.GeoDataFrame({"id": range(n_points), "d": distances_km, "azimuth": azimuth_deg}, geometry=points_utm, crs=utm_crs)
    gdf = gdf_utm.to_crs("EPSG:4326")

    gdf["zone"] = 0
    ct_codes = []
    with rasterio.open(tif_path) as ds:
        band = ds.read(1)
        for geom in gdf.geometry:
            row, col = ds.index(geom.x, geom.y)
            if 0 <= row < ds.height and 0 <= col < ds.width:
                ct_codes.append(int(band[row, col]))
            else:
                ct_codes.append(254)

    gdf["ct"] = ct_codes
    gdf["Ct"] = gdf["ct"].map(lambda c: lcm10_to_ct.get(c, 2))
    gdf["R"] = gdf["Ct"].map(lambda ct: ct_to_r.get(ct, 0))

    # Sample elevation from VRT file
    h = []
    cache_dir = elevation.CACHE_DIR
    vrt_path = Path(cache_dir) / "SRTM1" / "SRTM1.vrt"
    
    if vrt_path.exists():
        try:
            with rasterio.open(str(vrt_path)) as dem:
                for geom in gdf.geometry:
                    row, col = dem.index(geom.x, geom.y)
                    if 0 <= row < dem.height and 0 <= col < dem.width:
                        z = float(dem.read(1)[int(row), int(col)])
                    else:
                        z = 0.0
                    h.append(z)
        except Exception as e:
            print(f"Warning: Could not read DEM VRT ({e}), using 0 elevation")
            h = [0.0] * len(gdf)
    else:
        # Fallback: use zero elevation if VRT not available
        print(f"Warning: DEM VRT not found at {vrt_path}, using 0 elevation")
        h = [0.0] * len(gdf)

    gdf["h"] = h
    return gdf

## Generate Receivers and Fetch Land Cover

In [6]:
def generate_receivers_radial_multi(
    tx,
    distances_km: Iterable[Union[int, float]],
    azimuths_deg: Iterable[Union[int, float]],
    include_tx_point: bool = False,
):
    """Generate receivers on multiple rings around transmitter."""
    tx_gdf = gpd.GeoDataFrame(
        {"tx_id": [tx.tx_id]},
        geometry=[Point(tx.lon, tx.lat)],
        crs="EPSG:4326",
    )
    utm_crs = tx_gdf.estimate_utm_crs()
    tx_utm = tx_gdf.to_crs(utm_crs)
    tx_pt = tx_utm.geometry.iloc[0]

    rows = []
    rx_id = 1

    if include_tx_point:
        rows.append({
            "tx_id": tx.tx_id,
            "rx_id": 0,
            "distance_km": 0.0,
            "geometry": Point(tx.lon, tx.lat),
        })

    for d_km in distances_km:
        radius_m = float(d_km) * 1000.0
        for az in azimuths_deg:
            theta = math.radians(float(az))
            dx = radius_m * math.sin(theta)
            dy = radius_m * math.cos(theta)
            rx_utm = Point(tx_pt.x + dx, tx_pt.y + dy)
            rx_ll = gpd.GeoSeries([rx_utm], crs=utm_crs).to_crs("EPSG:4326").iloc[0]
            rows.append({
                "tx_id": tx.tx_id,
                "rx_id": rx_id,
                "distance_km": float(d_km),
                "azimuth_deg": float(az),
                "geometry": rx_ll,
            })
            rx_id += 1

    return gpd.GeoDataFrame(rows, geometry="geometry", crs="EPSG:4326")

# Generate receiver points
receivers = generate_receivers_radial_multi(tx, distances, azimuths, include_tx_point=True)
print(f"Generated {len(receivers)} receiver points")

Generated 793 receiver points


In [7]:
# Fetch and cache land cover from Sentinel Hub
print()
print("===== FETCH LAND COVER FROM SENTINEL HUB =====")
print()
client_id, client_secret = resolve_credentials(
    fallback_id=SH_CLIENT_ID,
    fallback_secret=SH_CLIENT_SECRET,
    verbose=True
)

lat = CONFIG['TRANSMITTER']['latitude']
lon = CONFIG['TRANSMITTER']['longitude']
buffer_m = CONFIG['SENTINEL_HUB']['buffer_m']
chip_px = CONFIG['SENTINEL_HUB']['chip_px']
year = CONFIG['SENTINEL_HUB']['year']

out_tif = api_data_dir / f"lcm10_{lat}_{lon}_{year}_buf{buffer_m}m_{chip_px}px.tif"

# Only fetch if not cached
if not out_tif.exists():
    print(f"Fetching land cover for ({lat}, {lon})...")
    code, chip = landcover_at_point(
        client_id, client_secret,
        lat, lon,
        token_url=TOKEN_URL,
        process_url=PROCESS_URL,
        collection_id=COLLECTION_ID,
        year=year,
        buffer_m=buffer_m,
        chip_px=chip_px,
        save_path=str(out_tif),
    )
    print(f"✓ Successfully saved GeoTIFF: {out_tif}")
else:
    print(f"✓ Using cached GeoTIFF: {out_tif}")


===== FETCH LAND COVER FROM SENTINEL HUB =====

  ✓ Found credentials in config file
✓ Using cached GeoTIFF: /Users/oz/Documents/mst_gis/data/intermediate/api_data/lcm10_9.345_-13.40694_2020_buf11000m_734px.tif


## Extract Profiles and Export

In [8]:
import time

# KEY OPTIMIZATION: Seed elevation ONCE before loop (not 36 times)
print("Seeding elevation data...")
total_start = time.time()
seed_start = time.time()
try:
    bounds = [tx_lon - 0.1, tx_lat - 0.1, tx_lon + 0.1, tx_lat + 0.1]
    elevation.seed(bounds=bounds, max_download_tiles=9)
    seed_time = time.time() - seed_start
    print(f"✓ Elevation data ready ({seed_time:.2f}s)\n")
except Exception as e:
    print(f"Warning: {e}\n")
    seed_time = 0

rows = []
tif_path_str = str(out_tif)
iter_times = []

for i, az in enumerate(azimuths):
    print(f"[{i+1}/{len(azimuths)}] Processing azimuth {az}°...", end=" ", flush=True)
    iter_start = time.time()
    
    try:
        gdf = generate_profile_points(
            tx_lon, tx_lat,
            max_distance_km,
            n_points,
            azimuth_deg=az,
            tif_path=tif_path_str,
            lcm10_to_ct=CONFIG['LCM10_TO_CT'],
            ct_to_r=CONFIG['CT_TO_R'],
            zones_path=None,
        )
        
        geom_0 = gdf.geometry.iloc[0]
        geom_last = gdf.geometry.iloc[-1]
        phi_t, lam_t = float(geom_0.y), float(geom_0.x)
        phi_r, lam_r = float(geom_last.y), float(geom_last.x)
        
        rows.append({
            "f": CONFIG['P1812']['frequency_ghz'],
            "p": CONFIG['P1812']['time_percentage'],
            "d": [round(v, 3) for v in gdf["d"].tolist()],
            "h": [int(round(v)) if v else 0 for v in gdf["h"].tolist()],
            "R": gdf["R"].tolist(),
            "Ct": gdf["Ct"].tolist(),
            "zone": gdf["zone"].tolist(),
            "htg": CONFIG['TRANSMITTER']['antenna_height_tx'],
            "hrg": CONFIG['TRANSMITTER']['antenna_height_rx'],
            "pol": CONFIG['P1812']['polarization'],
            "phi_t": phi_t,
            "phi_r": phi_r,
            "lam_t": lam_t,
            "lam_r": lam_r,
        })
        iter_time = time.time() - iter_start
        iter_times.append(iter_time)
        print(f"✓ ({iter_time:.2f}s)")
    
    except Exception as e:
        print(f"✗ ({type(e).__name__}: {str(e)[:60]})")
        iter_times.append(0)
        continue

total_time = time.time() - total_start
avg_time = sum(iter_times) / len(iter_times) if iter_times else 0
print(f"\nTiming Summary:")
print(f"  Seed time: {seed_time:.2f}s")
print(f"  Avg iteration: {avg_time:.2f}s")
print(f"  Total loop: {sum(iter_times):.2f}s")
print(f"  Total time: {total_time:.2f}s\n")

df_all = pd.DataFrame(rows)
output_path = profiles_dir / f"paths_oneTx_manyRx_{max_distance_km}km.csv"
df_all.to_csv(output_path, sep=";", index=False, decimal=".")

print()
print(f"✓ Saved {len(df_all)} profiles to {output_path}")

Seeding elevation data...
make: Nothing to be done for `download'.
make: Nothing to be done for `all'.
✓ Elevation data ready (0.04s)

[1/36] Processing azimuth 0°... ✓ (4.76s)
[2/36] Processing azimuth 10°... ✓ (4.72s)
[3/36] Processing azimuth 20°... ✓ (4.74s)
[4/36] Processing azimuth 30°... ✓ (4.70s)
[5/36] Processing azimuth 40°... ✓ (4.70s)
[6/36] Processing azimuth 50°... ✓ (4.71s)
[7/36] Processing azimuth 60°... ✓ (4.70s)
[8/36] Processing azimuth 70°... ✓ (4.71s)
[9/36] Processing azimuth 80°... ✓ (4.72s)
[10/36] Processing azimuth 90°... ✓ (4.69s)
[11/36] Processing azimuth 100°... ✓ (4.73s)
[12/36] Processing azimuth 110°... ✓ (4.71s)
[13/36] Processing azimuth 120°... ✓ (4.73s)
[14/36] Processing azimuth 130°... ✓ (4.72s)
[15/36] Processing azimuth 140°... ✓ (4.73s)
[16/36] Processing azimuth 150°... ✓ (4.75s)
[17/36] Processing azimuth 160°... ✓ (4.75s)
[18/36] Processing azimuth 170°... ✓ (4.75s)
[19/36] Processing azimuth 180°... ✓ (4.77s)
[20/36] Processing azimuth 190

## Summary

**Phase 1 Complete**:
- Configuration consolidation with CONFIG dict
- Helper functions for credentials and profile extraction
- Real terrain profiles with Sentinel Hub land cover + cached elevation data
- 36 profiles exported to CSV

Next: Run batch_processor.py to process profiles through P.1812-6 model.